In [1]:
!git clone https://bitbucket.org/kaamanita/blackboard-treebank.git

Cloning into 'blackboard-treebank'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 235 (delta 191), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (235/235), 68.27 MiB | 2.60 MiB/s, done.
Resolving deltas: 100% (191/191), done.


In [2]:
import pandas as pd
from pathlib import Path
import os

In [3]:
train = Path(os.path.join('.','blackboard-treebank','thai10'))

In [4]:
train_files = [f for f in train.iterdir() if not f.name.startswith('.')]
len(train_files)

25

In [5]:
def readfile(path):
    _data = []
    with open(path,'r',encoding="utf-8-sig") as f:
        _temp = [i for i in f.read().split("\n\n") if i != '']
    for j in _temp:
        _temp_data = []
        bad_data = False
        for i in j.split("\n"):
            if i.startswith('#') == False and not bad_data:
                _t = i.strip().split("\t")
                if _t[1]!="$$":
                    if _t[3]=="N":
                        _temp_data=[]
                        bad_data=True
                    else:
                        _temp_data.append((_t[1],_t[3]))
        if _temp_data!=[]:
            _data.append(_temp_data)
    return _data

In [6]:
train_data = []

In [7]:
for file in train_files:
    train_data.extend(readfile(file))

In [8]:
len(train_data)

130560

In [9]:
train_data[0]

[('ขณะนี้', 'NN'),
 ('พอ', 'AV'),
 ('จะ', 'AX'),
 ('ทราบ', 'VV'),
 ('ตัว', 'NN'),
 ('คนร้าย', 'NN'),
 ('แล้ว', 'AX')]

In [10]:
from pythainlp.tag.blackboard import post_process

In [12]:
train_data_ud = [post_process(i,to_ud=True) for i in train_data]

In [14]:
train_data[-1]

[('ที่', 'CC'),
 ('มี', 'VV'),
 ('ผล', 'NN'),
 ('ใน', 'PS'),
 ('การ', 'FX'),
 ('ยับยั้ง', 'VV')]

In [13]:
train_data_ud[-1]

[('ที่', 'CCONJ'),
 ('มี', 'VERB'),
 ('ผล', 'NOUN'),
 ('ใน', 'ADP'),
 ('การ', 'NOUN'),
 ('ยับยั้ง', 'VERB')]

In [15]:
from collections import defaultdict

In [16]:
pos_dict = defaultdict(list)

In [17]:
for i in train_data_ud:
    _word= [k for k,_ in i]
    _pos = '/'.join([k for _,k in i])
    pos_dict[_pos].append(_word)

In [19]:
temp="จงสร้างประโยคตามโครงสร้าง {pos}:"

In [43]:
inputs=[]
targets=[]

In [44]:
for i in sorted(list(pos_dict.keys())):
    if i.count("/")<1:
        continue
    for k in pos_dict[i]:
        inputs.append(temp.format(pos=i))
        targets.append(''.join(k))

In [45]:
inputs[-1],targets[-1]

('จงสร้างประโยคตามโครงสร้าง VERB/VERB/VERB/VERB/VERB/VERB/VERB/ADV/VERB/NOUN:',
 'โกรธคือโง่โมโหคือบ้าอย่าไปถือสาคน')

In [46]:
df_train = {
    "inputs":inputs,
    "targets":targets,
}

In [47]:
id=0
print(df_train["inputs"][id])
print(df_train["targets"][id])

จงสร้างประโยคตามโครงสร้าง ADJ/ADJ/AUX/VERB/VERB/NOUN/VERB/ADP/NOUN/ADV:
ก่อนอื่นต้องขอแสดงความยินดีกับยานินาด้วย


In [48]:
from datasets import Dataset, DatasetDict

In [49]:
ds = DatasetDict()

In [50]:
ds['train'] = Dataset.from_dict(df_train)

In [51]:
ds

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 130454
    })
})

In [52]:
ds.push_to_hub("pythainlp/blackboard_treebank_prompt",private=False)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]